## CNN: Fine-Tunning com HDC

### Imports

In [1]:
from modules import utils, globals
import torch
from modules import encoders
from binhd.classifiers import BinHD
from modules.cifake import Cifake
import wisardpkg as wp


In [2]:
# import kagglehub

# # # Download latest version
# path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

# print("Path to dataset files:", path)

### Carregando o dataset pré-treinado

In [3]:
model = utils.load_model_from_file("resnet18_cifake_finetuned_float32.pth")
print("Modelo pré-treinado carregado.")
print(model)

/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Modelo 'resnet18_cifake_finetuned_float32.pth' carregado para avaliação de desempenho.
Modelo pré-treinado carregado.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn

In [4]:
import os
import random
from torch.utils.data import Subset
from torchvision import models, transforms

import torchvision


transform = transforms.Compose([
                                     transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])
                  ])

train_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'train'),
    transform=transform
)

test_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'test'),
    transform=transform
)

# Cria subsets para testar a lógica do modelo com um número menor do dataset (descomentar para usar)
subset_train_indices = list(range(globals.NUM_SAMPLES_TRAIN_DEBUGGER))
subset_test_indices = list(range(globals.NUM_SAMPLES_TEST_DEBUGGER))

total_train_samples = len(train_dataset)
num_train_to_select = min(globals.NUM_SAMPLES_TRAIN_DEBUGGER, total_train_samples)
subset_train_indices = random.sample(range(total_train_samples), num_train_to_select)

# Pega uma amostra aleatória de índices para o teste
total_test_samples = len(test_dataset)
num_test_to_select = min(globals.NUM_SAMPLES_TEST_DEBUGGER, total_test_samples)
subset_test_indices = random.sample(range(total_test_samples), num_test_to_select)

train_dataset_debugger = Subset(train_dataset, subset_train_indices)
test_dataset_debugger = Subset(test_dataset, subset_test_indices)

train_loader = torch.utils.data.DataLoader(
    train_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])



In [5]:
all_features = []
all_labels = []

with torch.no_grad():
    for k, batch in enumerate(train_loader):
        print(f'\r{k+1}/{len(train_loader)}', end='', flush=True) 

        dado, rotulo = batch
        dado = dado.to(globals.DEVICE)
        rotulo = rotulo.to(globals.DEVICE)
        # Extrair features
        features = feature_extractor(dado)

        # Achatar (flatten) o tensor para (batch_size, num_features)
        features = features.view(features.size(0), -1)

        all_features.append(features.cpu())
        all_labels.append(rotulo)

print(all_features)
print(all_labels)

125/125[tensor([[1.5557, 2.2698, 1.8055,  ..., 2.0813, 0.0148, 0.4967],
        [2.6738, 0.7865, 1.0641,  ..., 0.4410, 0.7847, 1.4983],
        [1.0609, 1.5785, 2.1595,  ..., 0.7590, 0.4665, 0.7838],
        ...,
        [0.4011, 0.4422, 1.3688,  ..., 0.7283, 0.8392, 3.1221],
        [0.8407, 0.2369, 0.8617,  ..., 0.1402, 2.0098, 0.3161],
        [2.2442, 0.0627, 1.3959,  ..., 0.0525, 1.2472, 1.0323]]), tensor([[1.6124, 0.7833, 0.8377,  ..., 2.4848, 1.5769, 1.2740],
        [0.2332, 0.8346, 1.2927,  ..., 0.1444, 1.1281, 1.4850],
        [1.0662, 0.5812, 0.7848,  ..., 2.2183, 1.7169, 0.1327],
        ...,
        [1.8071, 0.3624, 2.4305,  ..., 1.6517, 0.1952, 1.8599],
        [0.7514, 0.8614, 0.3404,  ..., 0.3403, 0.3598, 0.5678],
        [3.3014, 1.9424, 1.3349,  ..., 0.3573, 0.2039, 1.6795]]), tensor([[1.1486, 0.3619, 0.6904,  ..., 0.0663, 2.0284, 1.4160],
        [0.9415, 0.5028, 0.1332,  ..., 0.5684, 1.2216, 0.1004],
        [1.4129, 0.2164, 0.1148,  ..., 0.1017, 1.1027, 0.4330],
  

In [6]:
import numpy as np

features_array = np.concatenate(all_features, axis=0)
labels_array = np.concatenate(all_labels, axis=0)

print(labels_array)


np.save('features.npy', features_array)
np.save('labels.npy', labels_array)

print("Features salvas em 'features.npy' e labels em 'labels.npy")

[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1
 1 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1 1
 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1
 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 0 0
 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0
 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1
 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0
 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 1 0
 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0
 0 0 1 1 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0
 0 0 0 1 1 1 0 0 1 0 0 0 

In [7]:
import pandas as pd


cifake = Cifake()
# Verifica as amostras e classes
print(f"Número de samples: {len(cifake.samples)}")
print(f"Classes encontradas: {cifake.features}")

Número de samples: 1000
Classes encontradas:        feat_0    feat_1    feat_2    feat_3    feat_4    feat_5    feat_6  \
0    1.555747  2.269822  1.805546  1.323721  0.833279  1.639890  0.265051   
1    2.673775  0.786515  1.064145  1.086338  0.734453  1.542318  1.773336   
2    1.060860  1.578544  2.159520  0.208679  0.231763  1.968023  2.870025   
3    0.227908  0.712186  3.134782  0.833350  1.491250  1.702767  0.726371   
4    1.002929  1.987897  0.991263  1.117672  3.303423  0.714118  1.296525   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.243292  0.403324  1.779496  0.717898  1.502337  0.993103  1.639076   
996  0.054340  0.206603  0.689380  0.481884  1.334408  0.960277  2.506159   
997  1.328808  0.381757  1.640507  1.636695  1.100611  0.812444  1.694885   
998  0.190977  0.192262  2.009255  0.718562  0.698256  0.838255  0.155917   
999  0.880879  0.937719  1.513298  1.212510  0.592163  0.848966  0.404383   

       feat_7    feat_8    fea

In [8]:
min_val, max_val = cifake.get_min_max_values()
print(min_val, max_val)

0.0 8.557672500610352


In [9]:
dimension = 1000
num_levels = 500
low = min_val
high = max_val
oper = "bind"

In [10]:
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder

from modules.encoders import RecordEncoder

X = cifake.features
print(X.shape)
print(X.dtypes.unique())

y = cifake.labels
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))


(1000, 512)
[dtype('float32')]


In [11]:
model = BinHD(dimension, cifake.num_classes)
print(X.dtypes)


feat_0      float32
feat_1      float32
feat_2      float32
feat_3      float32
feat_4      float32
             ...   
feat_507    float32
feat_508    float32
feat_509    float32
feat_510    float32
feat_511    float32
Length: 512, dtype: object


In [12]:
record_encoder = RecordEncoder(
            out_features=dimension,
            size=X.shape[1], 
            levels=num_levels,
            low=low,
            high=high
        )


In [ ]:
y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)

def run_encoders(X, device):
    # Garante que os rótulos estão no formato tensor e no dispositivo correto
    y_encoded_tensor = torch.tensor(y_encoded).to(device)

    # Lista para armazenar os vetores codificados por batch
    encoded_batches = []

    # Coloca o encoder no modo de avaliação e no dispositivo correto
    record_encoder.to(device)
    record_encoder.eval()

    # Número total de amostras
    num_samples = len(X)

    with torch.no_grad():
        for start_idx in range(0, num_samples, globals.BATCH_SIZE):
            end_idx = min(start_idx + globals.BATCH_SIZE, num_samples)

            # Seleciona um batch dos dados e converte para float32
            x_batch_np = X.iloc[start_idx:end_idx].values.astype(np.float32)

            # Converte para tensor e move para o dispositivo
            x_batch_tensor = torch.tensor(x_batch_np).to(device)

            # Codifica usando o encoder (mantendo a lógica original)
            encoded = record_encoder(x_batch_tensor)

            # Armazena o resultado no CPU
            encoded_batches.append(encoded.cpu())

            # Progresso
            print(f"\rProcessando amostras {start_idx} até {end_idx}", end='', flush=True)

    print("\nCodificação completa.")

    # Junta todos os batches em um único tensor
    encoded_all = torch.cat(encoded_batches, dim=0)

    return encoded_all



/tmp/ipykernel_71644/1262757083.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)


: 

In [ ]:
from sklearn.model_selection import train_test_split

X_record_encoder = run_encoders(X, globals.DEVICE)
labels = torch.tensor(y).to(globals.DEVICE)

X_train, X_test, y_train, y_test = train_test_split(X_record_encoder, labels, test_size=0.3, random_state = 0)

print(X_train, y_train)

1/125

In [ ]:
from sklearn.metrics import accuracy_score

with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test.to(torch.int8))  
    acc = accuracy_score(predictions, y_test)
    print("BinHD Record Encoder: Accuracy = ", acc)


BinHD Record Encoder: Accuracy =  0.3333333333333333
